# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
# Import API key
from api_keys import g_key

import warnings
warnings.filterwarnings('ignore')

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv", index_col=0)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,56.28,76,100,5.99,CA,1628233763
1,Longyearbyen,78.2186,15.6401,46.24,66,75,2.30,SJ,1628233764
2,Antalaha,-14.9003,50.2788,74.79,61,51,10.67,MG,1628233764
3,Menongue,-14.6585,17.6910,64.62,21,0,6.17,AO,1628233764
4,Bredasdorp,-34.5322,20.0403,56.28,91,89,4.38,ZA,1628233765


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#cities_short = cities_df[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70)], 
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 75) & 
               (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780,
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827,
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860,
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862,
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873,
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885,
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934,


In [14]:
params = {
    "radius": 5000,
    "types": "hotel",
    #"keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    #pprint.pprint(hotel_name)
    
#     print(index)
#     print(row)

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [15]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780,Colônia Velha
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827,Bussolengo
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860,Vilyuysk
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862,Kintinku
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873,Geraldton
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885,Salym
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934,Moroni


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map


# Display figure
